# 영업 성공 여부 분류 경진대회

In [ ]:
envs = ["colab_drive", "colab_local", "colab_home", "colab_gaeun"]
coding_env = envs[2]

if coding_env == "colab_drive" :
  from google.colab import drive
  drive.mount('/content/drive')

## 1. 데이터 확인

In [ ]:
!pip install imblearn
!pip install pycountry
!pip install seaborn

### 필수 라이브러리

In [ ]:
import pandas as pd
import numpy as np
import re
import pycountry
import torch
from sklearn.metrics import (
    accuracy_score,
    confusion_matrix,
    f1_score,
    precision_score,
    recall_score,
)
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

import matplotlib.pyplot as plt
import numpy as np
from matplotlib.colors import ListedColormap

import seaborn as sns


### 데이터 셋 읽어오기

In [ ]:
#colab에서 google drive 마운트 시
if coding_env == "colab_drive":
    df_train = pd.read_csv('/content/drive/MyDrive/해커톤 2402/train.csv')
    df_test = pd.read_csv('/content/drive/MyDrive/해커톤 2402/submission.csv')
#colab에서 train.csv와 submission.csv 데이터 불러온 후 사용
elif coding_env == "colab_local":
    df_train = pd.read_csv("/content/train.csv") # 학습용 데이터
    df_test = pd.read_csv("/content/submission.csv") # 테스트 데이터(제출파일의 데이터)
elif coding_env == "colab_home":
    df_train = pd.read_csv("./content/train.csv") # 학습용 데이터
    df_test = pd.read_csv("./content/submission.csv") # 테스트 데이터(제출파일의 데이터)

#모든 칼럼들
all_columns = list(df_train.columns)

df_all = pd.concat([df_train[all_columns], df_test[all_columns]])
df_all = df_all.reset_index(drop=True)

print('df_train.shape : ', df_train.shape)
print('df_test.shape : ', df_test.shape)
print('df_all.shape : ', df_all.shape)

df_train.shape :  (59299, 29)
df_test.shape :  (5271, 30)
df_all.shape :  (64570, 29)


### 데이터 형태 파악

In [ ]:
# prompt: graphs for analyse the df_train dataframe and df_test dataframe

import matplotlib.pyplot as plt
# df_train 데이터프레임 분석을 위한 그래프
# 1. 각 컬럼의 데이터 타입 분포
df_train.dtypes.value_counts().plot(kind='bar')
plt.title('Data types in df_train')
plt.xlabel('Data type')
plt.ylabel('Count')
plt.show()

# 2. 결측치 분포
df_train.isnull().sum().sort_values(ascending=False).plot(kind='bar')
plt.title('Missing values in df_train')
plt.xlabel('Column')
plt.ylabel('Count')
plt.show()

# 3. 타겟 변수 분포
df_train['is_converted'].value_counts().plot(kind='bar')
plt.title('Target variable distribution in df_train')
plt.xlabel('Target value')
plt.ylabel('Count')
plt.show()

# df_test 데이터프레임 분석을 위한 그래프
# 1. 각 컬럼의 데이터 타입 분포
df_test.dtypes.value_counts().plot(kind='bar')
plt.title('Data types in df_test')
plt.xlabel('Data type')
plt.ylabel('Count')
plt.show()

# 2. 결측치 분포
df_test.isnull().sum().sort_values(ascending=False).plot(kind='bar')
plt.title('Missing values in df_test')
plt.xlabel('Column')
plt.ylabel('Count')
plt.show()

# df_train 그래프
df_train.plot(kind="hist", figsize=(10, 8), bins=16, alpha=0.5)
plt.title("Histogram of df_train")
plt.xlabel("Value")
plt.ylabel("Frequency")
plt.show()

# df_test 그래프
df_test.plot(kind="hist", figsize=(10, 8), bins=16, alpha=0.5)
plt.title("Histogram of df_test")
plt.xlabel("Value")
plt.ylabel("Frequency")
plt.show()

#df_train 데이터프레임 기초통계량
print(df_train.describe())

#df_test 데이터프레임 기초통계량
print(df_test.describe())

#df_train 데이터프레임 칼럼별 고유값 개수
print(df_train.nunique())

#df_test 데이터프레임 칼럼별 고유값 개수
print(df_test.nunique())

# ### 그래프로 분석
# 1. 변수별 분포
# 2. 변수별 상관관계

# 변수별 분포
df_train.hist(figsize=(10, 10))
plt.show()

# 변수별 상관관계
corr_matrix = df_train.corr()
plt.figure(figsize=(10, 10))
sns.heatmap(corr_matrix, annot=True)
plt.show()

corr_matrix = df_test.corr()
plt.figure(figsize=(10, 10))
sns.heatmap(corr_matrix, annot=True)
plt.show()

### 함수 실행 여부 True False 설정

In [ ]:
#데이터 정제 True False
customer_country_cleaning = True
customer_country_1_cleaning = True
customer_type_cleaning = True
customer_job_cleaning = False
inquiry_type_cleaning = True
product_category_cleaning = True
product_subcategory_cleaning = True
product_modelname_cleaning = True
customer_position_cleaning = False
expected_timeline_cleaning = True

## 2. 데이터 전처리

###데이터 정제

#### customer country 정제

In [ ]:
#customer country 정제
if customer_country_cleaning :
  for index, data in enumerate(df_all['customer_country']):
    #customer country 데이터가 비어있으면 float 타입으로 검색됨, float인지 아닌지 검사
    if type(data) == type(float(1)) :
      df_all.loc[index, 'customer_country'] = 'no_country_data'
      continue

    #비어있지 않다면 스플릿 진행
    country = data.split('/')[-1]

    #모두 소문자화
    country = country.lower()

    #맨 앞에 공백문자가 있다면 제거
    try :
      if country[0] == ' ':
        country = country[1:]
    except :
      pass

    #만약 스플릿 된 문자열의 길이가 0이면 no_country_data로 컨버팅
    if len(country) == 0:
      df_all.loc[index, 'customer_country'] = 'no_country_data'
      continue

    #UAE 와 turkey 오류 해결
    if country == 'u.a.e':
      country = 'ARE'
    if country == 'turkey':
      country = 'TUR'

    #5개 숫자의 우편번호 = 미국
    is_post_US = re.compile('.*[0-9]{4,5}.*')
    #email address
    is_email = re.compile('.*[@].*[.].*')

    #Convert Country to alpha_3
    if is_post_US.match(country) != None or 'united states' in country:
        df_all.loc[index, 'customer_country'] = 'USA'
        continue
    elif is_email.match(country) != None:
        df_all.loc[index, 'customer_country'] = 'no_country_data'
        continue

    try :
        df_all.loc[index, 'customer_country'] = pycountry.countries.search_fuzzy(country)[0].alpha_3
        continue
    except :
        df_all.loc[index, 'customer_country'] = 'no_country_data'
        continue


  print(len(set(df_all['customer_country'])))

#### customer country 1 정제

In [ ]:
#customer country 1 정제
if customer_country_1_cleaning :
  for index, data in enumerate(df_all['customer_country.1']):
    #customer country 데이터가 비어있으면 float 타입으로 검색됨, float인지 아닌지 검사
    if type(data) == type(float(1)) :
      df_all.loc[index, 'customer_country.1'] = 'no_country1_data'
      continue

    #비어있지 않다면 스플릿 진행
    country = data.split('/')[-1]

    #모두 소문자화
    country = country.lower()

    #만약 스플릿 된 문자열의 길이가 0이면 No Data로 컨버팅
    if len(country) == 0:
      df_all.loc[index, 'customer_country.1'] = 'no_country1_data'
      continue

    #맨 앞에 공백문자가 있다면 제거
    try :
      if country[0] == ' ':
        country = country[1:]
    except :
      pass

    #UAE 와 turkey 오류 해결
    if country == 'u.a.e':
      country = 'ARE'
    if country == 'turkey':
      country = 'TUR'

    #5개 숫자의 우편번호 = 미국
    is_post_US = re.compile('.*[0-9]{4,5}.*')
    #email address
    is_email = re.compile('.*[@].*[.].*')


    #Convert Country to alpha_3
    if is_post_US.match(country) != None or 'united states' in country:
      df_all.loc[index, 'customer_country.1'] = 'USA'
      continue
    elif is_email.match(country) != None:
      df_all.loc[index, 'customer_country.1'] = 'no_country1_data'
      continue

    try :
      df_all.loc[index, 'customer_country.1'] = str(pycountry.countries.search_fuzzy(country)[0].alpha_3) + '1'
      continue
    except :
      df_all.loc[index, 'customer_country.1'] = 'no_country1_data'
      continue


  print(len(df_all['customer_country.1']))

#### inquiry_type 정제

In [ ]:
if inquiry_type_cleaning :
  if coding_env == "colab_local" :
    convert_inquiry_type_path = '/content/convert_inquiry_type.csv'
  elif coding_env == "colab_home" :
    convert_inquiry_type_path = './content/convert_inquiry_type.csv'
  convert_inquiry_type = pd.read_csv(convert_inquiry_type_path)
  convert_data = ''

  #print(df_all['inquiry_type'])

  for index, data in enumerate(df_all['inquiry_type']):
    try :
      convert_data = convert_inquiry_type.loc[convert_inquiry_type.value == data, ['convert']].values
      df_all['inquiry_type'].iloc[index] = convert_data[0][0]
    except :
      convert_data = 'no_data'
      df_all['inquiry_type'].iloc[index] = convert_data

  #print(df_all['inquiry_type'])

#### customer_type 정제

In [ ]:
if customer_type_cleaning :
  if coding_env == "colab_local" :
    convert_customer_type_path = '/content/convert_customer_type.csv'
  elif coding_env == "colab_home" :
    convert_customer_type_path = './content/convert_customer_type.csv'
  convert_customer_type = pd.read_csv(convert_customer_type_path)
  convert_data = ''

  #print(df_all['customer_type'])

  for index, data in enumerate(df_all['customer_type']):
    try :
      convert_data = convert_customer_type.loc[convert_customer_type.value == data, ['convert']].values
      df_all['customer_type'].iloc[index] = convert_data[0][0]
    except :
      convert_data = 'no_data'
      df_all['customer_type'].iloc[index] = convert_data

  #print(df_all['customer_type'])

#### product_category 정제

In [ ]:
if product_category_cleaning :
  if coding_env == "colab_local" :
    convert_product_category_path = '/content/convert_product_category.csv'
  elif coding_env == "colab_home" :
    convert_product_category_path = './content/convert_product_category.csv'
  convert_product_category = pd.read_csv(convert_product_category_path)
  convert_data = ''

  #print(df_all['product_category'])

  for index, data in enumerate(df_all['product_category']):
    try :
      convert_data = convert_product_category.loc[convert_product_category.value == data, ['convert']].values
      df_all['product_category'].iloc[index] = convert_data[0][0]
    except :
      convert_data = 'no_data'
      df_all['product_category'].iloc[index] = convert_data

  #print(df_all['product_category'])

#### product_subcategory 정제

In [ ]:
if product_subcategory_cleaning :
  product_subcategory_isnull = df_all['product_subcategory'].isnull()

  for index, _ in enumerate(df_all['product_subcategory']):
    try :
      df_all['product_subcategory'].iloc[index] = product_subcategory_isnull.iloc[index]
    except :
      df_all['product_subcategory'].iloc[index] = False

<ipython-input-19-341e125cddc3>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_all['product_subcategory'].iloc[index] = product_subcategory_isnull.iloc[index]


#### product_modelname 정제

In [ ]:
if product_modelname_cleaning :
  product_modelname_isnull = df_all['product_modelname'].isnull()

  for index, _ in enumerate(df_all['product_modelname']):
    try :
      df_all['product_modelname'].iloc[index] = product_modelname_isnull.iloc[index]
    except :
      df_all['product_modelname'].iloc[index] = False

<ipython-input-20-1f7b0c846fb0>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_all['product_modelname'].iloc[index] = product_modelname_isnull.iloc[index]


#### lead_desc_length 정제

#### expected_timeline 정제

In [ ]:
if expected_timeline_cleaning :
  for index, data in enumerate(df_all['expected_timeline']):
    data = str(data).replace('_',' ').replace('-','~')
    if 'less than 3 months' in data :
      data = 'less than 3 months'
    elif data == 'less than 6 months':
      data = '3 months ~ 6 months'

    valid_data = set(df_test['expected_timeline'])
    if data in valid_data:
      df_all['expected_timeline'].iloc[index] = data
    else :
      df_all['expected_timeline'].iloc[index] = None


<ipython-input-48-42dd0a94ad0c>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_all['expected_timeline'].iloc[index] = data
<ipython-input-48-42dd0a94ad0c>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_all['expected_timeline'].iloc[index] = None


### 데이터 인코딩

In [ ]:

  label_columns = [
      "customer_country",
      "business_unit",
      "customer_type",
      "enterprise",
      "customer_job",
      "inquiry_type",
      "product_category",
      "product_subcategory",
      "product_modelname",
      "customer_country.1",
      "customer_position",
      "response_corporate",
      "expected_timeline",
      "business_area",
      "business_subarea",
      "lead_owner"
  ]
  # 원 핫 인코딩 할 칼럼들
  onehot_columns = []
elif mode == 2:
  # 레이블 인코딩할 칼럼들
  label_columns = [
      "enterprise",
      "customer_job",
      "product_subcategory",
      "product_modelname",
      "customer_position",
      "response_corporate",
      "expected_timeline"
  ]
  # 원 핫 인코딩 할 칼럼들
  onehot_columns = [
      "customer_country",
      "business_unit",
      "customer_type",
      "inquiry_type",
      "product_category",
      "customer_country.1",
      "business_area",
      "business_subarea"
  ]

#사용하지 않을 칼럼들
not_use_columns = [
    "customer_idx",
    "customer_position",
    "business_subarea",
    "lead_owner"
]

#### 레이블 인코딩

In [ ]:
def label_encoding(series: pd.Series) -> pd.Series:
    """범주형 데이터를 시리즈 형태로 받아 숫자형 데이터로 변환합니다."""

    my_dict = {}

    # 모든 요소를 문자열로 변환
    series = series.astype(str)

    for idx, value in enumerate(sorted(series.unique())):
        my_dict[value] = idx
    series = series.map(my_dict)

    return series

for col in label_columns:
    df_all[col] = label_encoding(df_all[col])

다시 학습 데이터와 제출 데이터를 분리합니다.

In [ ]:
#label encoded datas
for col in label_columns:
    df_train[col] = df_all.iloc[: len(df_train)][col]
    df_test[col] = df_all.iloc[len(df_train) :][col].reset_index(drop=True)

In [ ]:
#사용하지 않을 칼럼 제거
for col in not_use_columns:
  try :
    df_train = df_train.drop(col, axis=1)
  except :
    pass
  try :
    df_test = df_test.drop(col, axis=1)
  except :
    pass

### 학습, 검증 데이터 분리

In [ ]:
x_train, x_val, y_train, y_val = train_test_split(
    df_train.drop("is_converted", axis=1),
    df_train["is_converted"],
    test_size=0.2,
    shuffle=True,
    random_state=400,
)

## 3. 모델 학습

### 모델 성능 함수 정의

In [ ]:
def get_clf_eval(y_test, y_pred=None):
    confusion = confusion_matrix(y_test, y_pred, labels=[True, False])
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, labels=[True, False])
    recall = recall_score(y_test, y_pred)
    F1 = f1_score(y_test, y_pred, labels=[True, False])

    print("오차행렬:\n", confusion)
    print("\n정확도: {:.4f}".format(accuracy))
    print("정밀도: {:.4f}".format(precision))
    print("재현율: {:.4f}".format(recall))
    print("F1: {:.4f}".format(F1))

## 4. 제출하기

### 테스트 데이터 예측

In [ ]:
# 예측에 필요한 데이터 분리
x_test = df_test.drop(["is_converted", "id"], axis=1)
x_test = x_test.fillna(0)

In [ ]:
from lightgbm import LGBMClassifier

lr = 0.1
max_iter = 2000
boosting_type_lists = 'gbdt'
num_leaves_lists = 8
max_depth_lists = -1
learning_rate_lists = 0.01
n_estimators_lists = 1000
subsample_for_bin_lists = 1000
objective = 'binary'
class_weight_lists = {True:15, False:1}

lgbm = LGBMClassifier(subsample_for_bin = subsample_for_bin, n_estimators = n_estimators, learning_rate = learning_rate, boosting_type = boosting_type, max_depth = max_depth, num_leaves=num_leaves, n_jobs=-1, objective = objective, boost_from_average=True, verbosity = -1, is_unbalance = True)
lgbm.fit(x_train, y_train)

test_pred = lgbm.predict(x_test.fillna(0))
sum(test_pred) # True로 예측된 개수

204

### 제출 파일 작성

In [ ]:
# 제출 데이터 읽어오기 (df_test는 전처리된 데이터가 저장됨)
df_sub = pd.read_csv("/content/submission.csv")
df_sub["is_converted"] = test_pred

# 제출 파일 저장
df_sub.to_csv("/content/submission.csv", index=False)

**우측 상단의 제출 버튼을 클릭해 결과를 확인하세요**